In [29]:
#import all libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import zipfile
import logging

In [30]:
def get_links(input_url):
    pass #TODO

In [31]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [58]:
def get_fa_title(soup):
    titles = soup.select('.product-name strong')
    title_list = []
    for title in titles:
        title_list += [title.text]
    return title_list


In [61]:
def get_en_title(soup):
    titles = soup.select('.product-name-englishname')
    title_list = []
    for title in titles:
        title_list += [title.text]
    return title_list

In [67]:
def get_price(soup):
    prices = soup.select('.price-broken , .col-md-7 .price:nth-child(1)')
    price_list = []
    for price in prices:
        price_list += [price.text]
    return price_list  #TODO make the price integer

In [35]:
def get_discount(soup):
    pass #TODO

In [36]:
def get_score(soup):
    pass #TODO

In [37]:
def get_publisher(soup):
    pass #TODO

In [38]:
def get_author(soup):
    pass #TODO

In [39]:
def get_code(soup):
    soups=soup.select('tr:nth-child(1) td+ td')
    codes=[]
    for s in soups:
        codes.append(s.text)

    return codes

In [40]:
def get_isbn(soup):
    soups = soup.select('.product-table .ltr')
    isbn_numbers = []
    for inum in soups:
        cleaned_text = inum.text.strip()  
        cleaned_text = re.sub('[^0-9-]', '', cleaned_text)  
        isbn_numbers.append(cleaned_text)

    return isbn_numbers

In [41]:
def get_pages(soup):
    soups = soup.select('tr:nth-child(5) .rtl')
    pages=[]
    for i in soups:
        cleaned_text=i.text.strip()
        cleaned_text = re.sub('[^0-9-]', '', cleaned_text)
        pages.append(cleaned_text)
    return pages

In [42]:
def get_per_date(soup):
    pass #TODO

In [43]:
def get_ad_date(soup):
    pass #TODO

In [44]:
def get_size(soup):
    pass #TODO

In [45]:
def get_material(soup):
    pass #TODO

In [46]:
def get_series(soup):
    pass #TODO

In [47]:
def get_send_time(soup):
    pass

In [48]:
def get_summary(soup):
    try:
        summary = soup.select('.product-description')[0].text
    except Exception:
        summary = None
        logging.exception("This book has no summary!")
    return summary

In [49]:
def get_tags(soup):
    pass

In [66]:
#test cell  #TODO temp
links = ['https://www.iranketab.ir/book/270-gone-with-the-wind',
         'https://www.iranketab.ir/book/121475-foot-soldiers-do-not-reach-the-end',
         'https://www.iranketab.ir/book/121501-setaregan-e-sarnevesht',
         'https://www.iranketab.ir/book/119702-devil-in-a-blue-dress',
         'https://www.iranketab.ir/book/119533-saving-money']
for link in links:
    test = get_soup(link)
    print('farsi title; ', get_fa_title(test))
    print('english title: ', get_en_title(test))
    print('price: ', get_price(test))
    print('summary:\n', get_summary(test))
    print("----------------------------------")

farsi title;  ['کتاب بر باد رفته', 'برباد رفته', 'بر باد رفته', 'برباد رفته (۲ جلدی)', 'بر باد رفته', 'بر باد رفته', 'بر باد رفته', 'برباد رفته']
english title:  [' Gone with the Wind', ' Gone with the Wind', ' Gone with the Wind', ' Gone with the Wind', ' Gone with the Wind', ' Gone with the Wind', ' Gone with the Wind', ' Gone with the wind']


ValueError: invalid literal for int() with base 10: ' 480,000 '

In [22]:
data_list = []
url = "temp"  #TODO
links = get_links(url)

soups = get_soup(links)

for site_soup in soups:
    try:
        book_fa_title = get_fa_title(site_soup)
        book_en_title = get_en_title(site_soup)
        book_price = get_price(site_soup)
        book_discount = get_discount(site_soup)
        book_score = get_score(site_soup)
        book_publisher = get_publisher(site_soup)
        book_author = get_author(site_soup)
        book_code = get_code(site_soup)
        book_Isbn = get_isbn(site_soup)
        book_pages = get_pages(site_soup)
        book_publication_per_date = get_per_date(site_soup)
        book_publication_ad_date = get_ad_date(site_soup)
        book_size = get_size(site_soup)
        book_cover_material = get_material(site_soup)
        book_print_series = get_series(site_soup)
        book_earliest_send_time = get_send_time(site_soup)
        book_summary = get_summary(site_soup)
        book_tags = get_tags(site_soup)

        data = [book_fa_title, book_en_title, book_price, book_discount, book_score, book_publisher, book_author,
                book_code, book_Isbn, book_pages, book_publication_per_date, book_publication_ad_date, book_size,
                book_cover_material, book_print_series, book_earliest_send_time, book_summary, book_tags]  #TODO
        data_list += [data]
    except Exception:
        logging.exception("An error occurred")

MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?